In [ ]:
import mne
import pandas as pd
import numpy as np
import scipy.io
import scipy.signal as ss
import pyedflib
from scipy.signal import hilbert,butter, lfilter,welch
from scipy.integrate import simps
from meegkit.asr import ASR
from meegkit.utils.matrix import sliding_window
from mne.time_frequency import psd_array_multitaper
from neurodsp.utils.download import load_ndsp_data
sampling_rate = 1000

def butterBandpass(data, lower_limit_filter, upper_limit_filter, sampling_rate, order=4):
    """
    This func is for filtering signal between lower and upper bounds
    the methods are used from scipy.signal lib
    """
    nyquist_coeff = 0.5 * sampling_rate
    low_frequences_filter = lower_limit_filter / nyquist_coeff
    high_frequences_filter = upper_limit_filter / nyquist_coeff
    numerator_filter, denominator_filter = butter(order, 
                                                  [low_frequences_filter, high_frequences_filter],
                                                  btype='band')
    # based on numinator and denominator the filter signal ...                                            )
    filtered_signal = lfilter(numerator_filter, denominator_filter, data)
    return filtered_signal

def applyArtifactSubspaceReconstruction(raw, sfreq=sampling_rate, cutoff=2.5, 
                                        blocksize=100, win_len=0.5,
                                        win_overlap=0.66, max_dropout_fraction=0.1,
                                        min_clean_fraction=0.25, name='asrfilter', method='euclid',
                                        estimator='scm'):
    """
    Goal: this function removes artifact specially those related to EOG noises
    """
    h, w = raw.shape
    
    # t = int(raw.shape[1] / sfreq)
    # take asr ....
    asr = ASR(sfreq=sfreq, cutoff=cutoff, blocksize=blocksize, win_len=win_len,
        win_overlap=win_overlap, max_dropout_fraction=max_dropout_fraction,
        min_clean_fraction=min_clean_fraction, name=name, method=method,
        estimator=estimator)   
    # method='euclid'
    # train_idx = np.arange(0 * sfreq, t * sfreq, dtype=int)
    
    # short description ....
    _, sample_mask = asr.fit(raw)
    
    # Apply filter using sliding (non-overlapping) windows
    # name X, Y
    # description
    #sliding_window function converts our 2-D data(number of channels by number of recorded samples) dataset to a 3-D function(number of channels by time in second by sampling rate) 
    X = sliding_window(raw, window=int(sfreq), step=int(sfreq)) 
    Y = np.zeros_like(X)
    for i in range(X.shape[1]):
        Y[:, i, :] = asr.transform(X[:, i, :])
   # reshape to (n_chans, n_times)
    clean = Y.reshape(h, -1)
    return clean

In [ ]:
hh=np.array([32,33,34,35,36,38,37,38,39,40,41,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,67,
             68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98])
for i in range(len(hh)):
    raw= mne.io.read_raw_eeglab('D:\WM_openneuro_dataset\sub-0'+str(hh[i])+'_task-rest_eeg.set')
    all_events, all_event_id = mne.events_from_annotations(raw)
    my_events= mne.pick_events(all_events)
    epochs = mne.Epochs(raw,my_events, tmin=0.0, tmax=200.0, baseline=(0, 0))
    data=epochs.to_data_frame()
    data.drop('time', axis=1, inplace=True)
    data.drop('condition', axis=1, inplace=True)
    data.drop('epoch', axis=1, inplace=True)
    data.drop([0],axis=0,inplace=True)
    edata=data.to_numpy(dtype ='float32')
    clm,rw =data.shape
    eeg=np.reshape(edata,(rw,-1))
    filter_signal = butterBandpass(eeg, 1, 49, 1000)
    resmpl_EEG=np.empty((63,50000))
    for ll in range(63):
        resmpl_EEG[ll,:]=ss.resample(filter_signal[ll,:],50000)
    filter_asr_signal = applyArtifactSubspaceReconstruction(resmpl_EEG)
    np.save('s'+str(hh[i])+'2_clean_resting_state_64', filter_asr_signal)